In [5]:
import pandas as pd
import numpy as np
import os
import random

from tensorflow.keras.preprocessing.image import ImageDataGenerator

#import tensorflow as tf

from keras.layers import Dropout, Dense, Input, Lambda, MaxPooling2D, Conv2D, Flatten, GlobalAveragePooling2D
from keras.models import Sequential, Model
from keras.models import load_model
from keras import backend as K
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
import tensorflow as tf
from keras.applications.vgg16 import VGG16


In [9]:
# example of loading an image with the Keras API
from keras.preprocessing.image import load_img
# load the image
img = load_img('../data/train_data_set/'+train_files[0])
# report details about the image
print(type(img))
print(img.format)
print(img.mode)
print(img.size)
# show the image
img.show()

NameError: name 'train_files' is not defined

In [24]:
# example of converting an image with the Keras API
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
# load the image
img = load_img('data/train_data_set/'+train_files[0])
print(type(img))
# convert to numpy array
img_array = img_to_array(img)
print(img_array.dtype)
print(img_array.shape)
# convert back to image
img_pil = array_to_img(img_array)
print(type(img))

<class 'PIL.JpegImagePlugin.JpegImageFile'>
float32
(2048, 3072, 3)
<class 'PIL.JpegImagePlugin.JpegImageFile'>


In [25]:
img2 = img.rotate(90)
img2.show()

In [11]:
all_pairs = pd.read_csv("smoke_pairs.csv", delimiter=",")

In [12]:
all_pairs.head()

,Unnamed: 0,fPath1,fPath2,label,test_label
0,0,../data/train_data_set/20170625-BBM-bm-n-mobo/...,../data/train_data_set/20170625-BBM-bm-n-mobo/...,0,False
1,1,../data/train_data_set/20170625-BBM-bm-n-mobo/...,../data/train_data_set/20170625-BBM-bm-n-mobo/...,0,False
2,2,../data/train_data_set/20170625-BBM-bm-n-mobo/...,../data/train_data_set/20170625-BBM-bm-n-mobo/...,0,False
3,3,../data/train_data_set/20170625-BBM-bm-n-mobo/...,../data/train_data_set/20170625-BBM-bm-n-mobo/...,0,False
4,4,../data/train_data_set/20170625-BBM-bm-n-mobo/...,../data/train_data_set/20170625-BBM-bm-n-mobo/...,0,False


In [13]:
img_file = load_img(all_pairs.loc[random.randint(0, len(all_pairs))]['fPath1'])

In [14]:
img_file.size

(2048, 1536)

In [15]:
img2 = img_file.resize([224, 224])

In [31]:
img2.show()

In [16]:
def get_batch(data_df, batch_size, is_test):
    # initialize 2 empty arrays for the input batch
    pairs=[np.zeros((batch_size, 224, 224, 3)).astype("int32") for i in range(2)]
    
    # initialize vector for the targets - target is a 1 or 0 for match or not matched
    targets=np.zeros((batch_size,))
    
    i=0
    while i<batch_size:
        data = data_df.loc[random.randint(0, len(data_df))]
        if data['test_label']==is_test:
            pairs[0][i,:,:,:] = load_img(data['fPath1']).resize([2048, 1536])
            pairs[1][i,:,:,:] = load_img(data['fPath2']).resize([2048, 1536])
            label = data['label']
            i+=1
    return pairs, targets

In [ ]:
#generate batches:
#select a random file
#find the next file in the sqeuence (see if there is a common pattern to do this via the name))
#if there is not a next file, start over
#convert both files to keras images
#apply the same random transformation to both images
#apply a label
#return pairs, labels

#alternatively, draw random samples from a paired and labeled 
#pandas dataframe/numpy array. 
#Oceane's doesn't seem to be sequential, see what Will built, or build own


In [17]:
def generate_batches(data_df, batch_size, is_test):
    while True:
        pairs, targets = get_batch(data_df, batch_size, is_test)
        yield (pairs, targets)

In [18]:
train_gen = generate_batches(all_pairs, 32, False)

val_gen = generate_batches(all_pairs, 32, True)

In [10]:
#model architecture:

def get_twin_model():
    
    image_width=2048
    image_height=1536
    input_shape = (image_width, image_height, 3)
    
    # Define the tensors for the two input documents
    left_input = Input(input_shape)
    right_input = Input(input_shape)

    model = Sequential()
    model.add(Conv2D(64, (100,100), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (7,7), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (4,4), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (4,4), activation='relu'))
    model.add(Flatten())
    model.add(Dense(4096, activation='sigmoid'))
    
    
    # Generate the encodings (feature vectors) for the two documents
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid')(L1_distance)
    
    # Connect the inputs with the outputs
    twin_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return twin_net

In [11]:
model = get_twin_model()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 2048, 1536, 3 0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 2048, 1536, 3 0                                            
__________________________________________________________________________________________________
sequential_3 (Sequential)       (None, 4096)         42995777088 input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 4096)         0           sequential_3[1][0]               
          

In [32]:
model_name = "scratch_unscaledimage_batch32"

In [33]:
optimizer = Adam()
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

stopper = EarlyStopping(monitor='val_acc', patience=5)
checkpointer = ModelCheckpoint("checkpoints/{}_continued.h5".format(model_name), monitor='val_acc', save_best_only=True, period = 10)

In [34]:
#adjust training parameters
history = model.fit_generator(train_gen, steps_per_epoch = 300, \
                              epochs = 50, verbose = 1, \
#                               callbacks=[tensorboard, stopper, checkpointer], \
                              callbacks=[stopper, checkpointer], \
                              validation_data = val_gen, validation_steps = 100, \
                              use_multiprocessing = True)

Epoch 1/50
  1/300 [..............................] - ETA: 11:35:14 - loss: 0.6008 - acc: 0.8125

Process ForkPoolWorker-5:
Process ForkPoolWorker-6:
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/anaconda3/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/anaconda3/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/anaconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", li

Epoch 1/50


  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/anaconda3/lib/python3.6/site-packages/keras/utils/data_utils.py", line 626, in next_sample
    return six.next(_SHARED_SEQUENCES[uid])
  File "<ipython-input-17-07acb45e46a9>", line 3, in generate_batches
    pairs, targets = get_batch(data_df, batch_size, is_test)
  File "<ipython-input-16-34f09c2dafea>", line 13, in get_batch
    pairs[1][i,:,:,:] = load_img(data['fPath2']).resize([224, 224])
  File "/anaconda3/lib/python3.6/site-packages/PIL/Image.py", line 1782, in resize
    self.load()
  File "/anaconda3/lib/python3.6/site-packages/PIL/ImageFile.py", line 239, in load
    n, err_code = decoder.decode(b)
KeyboardInterrupt


Epoch 1/50


KeyboardInterrupt: 

In [ ]:
#fix filepath
model.save("../../dbfs/mnt/databricks-cc/katelyn/ManuscriptMatcher/{}_final.h5".format(model_name))